In [ ]:
#importing part 
import requests
from bs4 import BeautifulSoup
import urllib.parse
import mysql.connector
import pyodbc

In [ ]:
#function to get final links and insert into database table 
def get_links(URL):
    global inside_links
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, features='lxml') 
    inside_links= [link.get("href") for link in soup("a") if "citizen_html" in link.get("href")]
    mycursur = db.cursor()
    sql1 = "INSERT INTO master (link) VALUES (%s)"
    for itm in range(0,len(inside_links)):
        val = inside_links[itm]
        mycursur.execute(sql1, (val,))
        db.commit()  
    return (len(inside_links))

In [ ]:
db = mysql.connector.connect(
    host="localhost",
    user="jov",
    passwd="123456789",
    database="ana"
)
mycursor = db.cursor()

#Deleting  and creating a new table
sql1 = "DROP TABLE master"
mycursor.execute(sql1)
mycursor.execute("CREATE TABLE master(id INT AUTO_INCREMENT PRIMARY KEY, link VARCHAR(10000))")
mycursor.close()

#link fetching part
urle1 ="https://mnregaweb4.nic.in/netnrega/master_search1.aspx?flag=2&wsrch=msr&district_code=3420&state_name=JHARKHAND&district_name=BOKARO&short_name=JH&srch=20000&Digest=+aw4ZIV9SpyInYb9MphsnQ "
urle1_parts = list(urllib.parse.urlparse(urle1))
qery1 = dict(urllib.parse.parse_qsl(urle1_parts[4]))
qery1["srch"]=1
for srch in range(0,100):      
        urle1_parts[4] = urllib.parse.urlencode(qery1)
        URL=urllib.parse.urlunparse(urle1_parts)
        #calling function  
        c= get_links(URL)
        if c:
            present = 1
            qery1["srch"] += 1            
        else:          
            print("NO links present in here")
            break
print("I stopped getting you links")  

In [ ]:
#to display the links stored in table master 
cursor= db.cursor()
cursor.execute("SELECT * FROM master") 
# fetch all the matching rows 
result = cursor.fetchall() 
# loop through the rows
for row in result:
    print(row)
    print("\n")